In [2]:
import cv2
from tracker import *
import numpy as np
from matplotlib import pyplot as plt
import imutils 


In [3]:
# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("Stereo conveyor without occlusions.mp4")

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract Region of interest
    #frame = frame[300:900,400:1150]
    rotated = imutils.rotate_bound(frame, 15)
    rotated2=rotated[450:800,350:1250]
    roi = imutils.rotate_bound(rotated2, -15)
    
    #hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #hsv_channels = cv2.split(hsv)
    #rows = frame.shape[0]
    #cols = frame.shape[1]

    #for i in range(0, rows):
    #    for j in range(0, cols):
    #        h = hsv_channels[0][i][j]
    #        if h > 90 and h < 130:
    #            hsv_channels[2][i][j] = 255
    #        else:
    #            hsv_channels[2][i][j] = 0
    #roi=hsv_channels[2]

    # 1. Object Detection
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 20, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(mask, None, iterations=6)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    detections = []

    biggest_area=0
    best_cnt = None
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area>biggest_area:
            biggest_area=area
            best_cnt = cnt
    if biggest_area>2000:
        x, y, w, h = cv2.boundingRect(best_cnt)
        detections.append([x, y, w, h])
    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id = box_id
        x=x-25
        y=y-25
        w=w+50
        h=h+50
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)
        #cv2.rectangle(mask, (x, y), (x + w, y + h), (0, 255, 0), 3)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", thresh)

    key = cv2.waitKey(30)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # q will end the program
        break

cap.release()
cv2.destroyAllWindows()

{5: (565, 283)}
{5: (565, 283)}
{7: (748, 67)}
{13: (653, 337)}
{13: (653, 337)}
{13: (654, 337)}
{13: (657, 334)}
{15: (882, 338)}
{15: (873, 333)}
{15: (873, 324)}
{15: (870, 320)}
{16: (870, 297)}
{16: (870, 299)}
{17: (836, 241)}
{17: (836, 228)}
{17: (836, 214)}
{17: (829, 203)}
{17: (826, 195)}
{17: (824, 190)}
{17: (824, 190)}
{17: (843, 177)}
{17: (840, 170)}
{17: (839, 160)}
{17: (838, 151)}
{17: (838, 143)}
{17: (838, 142)}
{17: (838, 145)}
{17: (838, 154)}
{17: (840, 163)}
{17: (842, 173)}
{18: (805, 155)}
{18: (803, 156)}
{18: (802, 162)}
{18: (800, 162)}
{18: (797, 163)}
{18: (797, 163)}
{18: (795, 164)}
{18: (792, 166)}
{18: (791, 168)}
{18: (789, 180)}
{18: (787, 179)}
{18: (785, 180)}
{18: (783, 170)}
{18: (781, 180)}
{18: (779, 171)}
{18: (777, 171)}
{18: (776, 171)}
{18: (774, 172)}
{18: (772, 172)}
{18: (770, 172)}
{18: (768, 172)}
{18: (765, 172)}
{18: (762, 170)}
{18: (762, 171)}
{18: (759, 171)}
{18: (757, 172)}
{18: (755, 180)}
{18: (753, 180)}
{18: (750, 181)}
{

In [20]:
# VERSION 1
import cv2
cap = cv2.VideoCapture("Stereo conveyor without occlusions.mp4")
ret, frame = cap.read()
firstFrame = None
while ret:
    #Detecting movement
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    rotated = imutils.rotate_bound(gray, 15)
    rotated2=rotated[450:800,350:1250]
    roi = imutils.rotate_bound(rotated2, -15)
    if firstFrame is None:
        firstFrame = roi
        continue

    feat1 = cv2.goodFeaturesToTrack(roi, maxCorners=1000, qualityLevel=0.4, minDistance=10)
    feat2, status, error = cv2.calcOpticalFlowPyrLK(roi, firstFrame, feat1, None)

    for i in range(len(feat1)):
        f10=int(feat1[i][0][0])
        f11=int(feat1[i][0][1])
        f20=int(feat2[i][0][0])
        f21=int(feat2[i][0][1])
        #cv2.circle(frame, (f10, f11), 5, (0, 255, 0), -1)
        #cv2.line(frame, (f10,f11), (f20, f21), (0, 255, 0), 2)

        if ((abs(f10-f20))/(f10+f20) >= 0.05) or ((abs(f11-f21))/(f11+f21) >= 0.05): # 0.005
            
            #cv2.line(frame, (f10,f11), (f20, f21), (0, 255, 0), 2)
            cv2.circle(roi, (f10, f11), 5, (0, 255, 0), -1)
    cv2.imshow('image', roi)
    cv2.waitKey
    
    #firstFrame = gray
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyAllWindows()
cap.release()